In [2]:
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, SystemMessage
from langgraph.types import Command
from uuid import uuid4
load_dotenv()

def read_email_tool(email_id: str)->str:
    """Mock function to read email by its ID."""
    return f"Email content for ID: {email_id}"


def send_email_tool(recepient: str, subject: str, body: str)->str:
    """Mock function to send an email"""
    return f"Email sent to {recepient} with subject '{subject}'"

In [3]:
agent = create_agent(model="gpt-5-mini", tools=[read_email_tool, send_email_tool], checkpointer=InMemorySaver(), middleware=[
    HumanInTheLoopMiddleware(interrupt_on={"send_email_tool":{
        "allowed_decisions":["approve","edit","reject"]
    },"read_email_tool": False})
])

In [9]:
config = {"configurable": {"thread_id":str(uuid4())}}

#step 1
result = agent.invoke(
    {"messages":[HumanMessage(content="Send email to yawkyereh@gmail.com with subject 'Hello' and body 'How are you?'")]}, config
)


if "__interrupt__" in result:
    print("Should i proceed with the email?")
    print("Paused! Approving...")
    response = input("type 'approve' or 'reject' to proceed: ")
    result = agent.invoke(Command(resume={
        "decisions":[{"type":f"{response}"}]
    }),config=config)
    print(f"Result: {result['messages'][-1].content}")

Should i proceed with the email?
Paused! Approving...
Result: Done — I sent the email to yawkyereh@gmail.com with the subject "Hello" and the body "How are you?".
